## Декораторы

Отчасти по причине того, что в python функции — это объекты первого порядка, в нем распространены то, что в других языках называется функциями высшего порядка, т.е. функции, которые принимают в качестве параметра или возвращают другие функции. Для конкретного вида таких функций в python добавили специальный элемент языка — декораторы.

In [1]:
def announce(func):
    def wrap(x):
        print(f"Функция {func.__name__} вызвана с аргументом {x}")
        return func(x)
    return wrap

Функция announce внутри тела объявляет новую функцию wrap и тем самым создаёт объект этой функции. Когда этот объект создан, он сразу возвращается наружу. Функция wrap в свою очередь является оберткой: она возвращает результат вычисления обертываемой функции func, но перед этим делает дополнительные действия, а именно печатает имя функции (атрибут func.__name__) и аргумент, с которым вызывается функция. С этим же аргументом вызывается функция func.

Проверим работоспособность это функции. Для этого обернем функции синуса и косинуса.

In [2]:
from math import sin, cos

print(sin(0), cos(0))

sin = announce(sin)
cos = announce(cos)

print(sin(0), cos(0))

0.0 1.0
Функция sin вызвана с аргументом 0
Функция cos вызвана с аргументом 0
0.0 1.0


Видно, что обертывание функций sin и cos приводит к желаемому эффекту.

Такого вида функции и называют декораторами. При этом, т.к. такие функции часто применяются и к пользовательским функциям, то в python ввели специальный синтаксический сахар для их применения.

Вместо комбинации инструкций
```python
def my_function(x):
    pass

my_function = announce(my_function)
```
можно смело писать
```python
@announce
def my_function(x):
    pass
```
Они эквиваленты между собой с точки зрения синтаксиса.

Но у второго подхода есть ряд преимуществ, самый главный из которых, наверное, следующий. При первом подходе читатель анализирует тело функции, ещё не зная, что она потом будет задекорирована, т.е. модифицирована. При втором подходе декоратор встречается ещё до ключевого слова def и явно указывает на модификацию функции.

In [3]:
@announce
def square(x):
    return x * x

print(square(2))

Функция square вызвана с аргументом 2
4


In [7]:
def decorator_function(func):
    def wrapper():
        print(f'Функция обертка!')
        print(f'выполняем func')
        func()
        print(f'выходим из обертки')
    return wrapper

@decorator_function
def hello_world():
    print('Hello world!')

hello_world()

Функция обертка!
выполняем func
Hello world!
выходим из обертки


In [13]:
def decorator_function(iters, param = 1):
    def actual_decorator(func):
        def wrapper():
            print(f'Функция обертка!\n' * iters)
            print(f'выполняем func\n' * param)
            func()
            print(f'выходим из обертки\n' * iters)
        return wrapper
    return actual_decorator

@decorator_function(iters=3, param=4)
def hello_world():
    print('_' * 40)
    print('Hello world!')
    print('_' * 40)


hello_world()

Функция обертка!
Функция обертка!
Функция обертка!

выполняем func
выполняем func
выполняем func
выполняем func

________________________________________
Hello world!
________________________________________
выходим из обертки
выходим из обертки
выходим из обертки



In [4]:
from functools import wraps

def announce(func):
    @wraps(func)
    def wrap(x):
        print(f"Функция {func.__name__} вызвана с аргументом {x}")
        return func(x)
    return wrap

@announce
def square(x):
    return x * x

print(square.__name__)
print(square)

square
<function square at 0x000001B48DF7E3B0>


In [5]:
def announce(func):
    def wrap(*args, **kwargs):
        print(f"Функция {func.__name__} вызвана с позиционными аргументами {args}", end="")
        print(f" и именованными аргументами {kwargs}.")
        return func(*args, **kwargs)
    return wrap

@announce
def add(x, y):
    return x + y

print(add(42, 3.14))
print(add(7, y=2.71))

Функция add вызвана с позиционными аргументами (42, 3.14) и именованными аргументами {}.
45.14
Функция add вызвана с позиционными аргументами (7,) и именованными аргументами {'y': 2.71}.
9.71


In [6]:
# Пример: измеряющий время декоратор
# Рассмотрим вариант реализации декоратора, который при вызове декорируемой функции измеряет время,
# потребовавшееся на её выполнение и выводит его в стандартный поток вывода, прежде чем вернуть результат её вычисления.

from functools import wraps
from time import perf_counter


def timed(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t1 = perf_counter()
        result = func(*args, **kwargs)
        t2 = perf_counter()
        print(f"Вызов {func.__name__} занял {t2- t1} секунд, ", end="")
        print(f"(параметры {args}, {kwargs})")
        return result
    return wrap


@timed
def countdown(n):
    while n > 0:
        n -= 1


for n in [10_000, 100_000, 1_000_000, 10_000_000]:
    countdown(n)

Вызов countdown занял 0.0005114000014145859 секунд, (параметры (10000,), {})
Вызов countdown занял 0.005111499998747604 секунд, (параметры (100000,), {})
Вызов countdown занял 0.0468269999983022 секунд, (параметры (1000000,), {})
Вызов countdown занял 0.5043105000004289 секунд, (параметры (10000000,), {})


In [21]:
# Замеряем время выполнения функции

def benchmark(func):
    import time

    def wrapper(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        end = time.time()
        print(f'Время выполнения: {end - start} секунд.')
        return return_value
    return wrapper


@benchmark
def fetch_webpage(url: str) -> str:
    import requests
    get_webpage = requests.get(url)
    return get_webpage.text


webpage = fetch_webpage('https://www.google.kz/')
print(webpage)

Время выполнения: 0.3752923011779785 секунд.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="kk"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="4_Y_2XXHlxQIoq0xL9aquA">(function(){var _g={kEI:'PMyuZInILJuSkdUP05OC-Aw',kEXPI:'0,1359409,1709,4349,207,2415,2389,2316,383,246,5,1129120,1749,1196018,881,379843,16114,28684,22430,1362,12311,17588,4998,17075,35733,2711,889,1983,2891,12360,29877,791,30021,16106,230,20583,4,1528,2304,42127,11443,6652,9358,13243,6636,7596,1,8710,33444,2,39761,5679,1021,25046,6076,4568,6258,23417,1253,5835,19300,5016,2468,445,2,2,1,6960,19672,8155,7381,15970,873,19632,10,1920,9779,12415,30044,20198,928,19209,14,82,2914,17292,8377,24363,2266,764,6111,9705,1804,10472,2884,2495,6986,4255,6005,2171,5250,3537,3027,1632,7951,5909,586,543,3148,2753,2374,3438,1991,3358,2565,450,4501,3899,147